# Logistic distribution simulation

In [1]:
library(evd)
library(maxstablePCA)
library(ggplot2)
library(dplyr)
library(GGally)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2



### setup parameters and simulate data 

In [2]:
set.seed(1842024)
s <- 250

In [3]:
X1 <- rmvevd(10000, dep = .2, model = "log", d = 5)
X2 <- rmvevd(10000, dep = .5, model = "log", d = 5)
X3 <- rmvevd(10000, dep = .8, model = "log", d = 5)

In [4]:
X1trafo <- transform_unitpareto(X1)
X2trafo <- transform_unitpareto(X2)
X3trafo <- transform_unitpareto(X3)

# max-stable PCA for X1

In [5]:
summary(rowSums(X1trafo))

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
    5.00     6.84    10.10    48.94    19.97 50000.00 

In [6]:
length(which(rowSums(X1trafo) > s))

[1] 207

In [7]:
maxPCAX1P1 <- max_stable_prcomp(X1trafo, p = 1, s = s, n_initial_guesses = 15000)
maxPCAX1P2 <- max_stable_prcomp(X1trafo, p = 2, s = s, n_initial_guesses = 15000)
maxPCAX1P3 <- max_stable_prcomp(X1trafo, p = 3, s = s, n_initial_guesses = 15000)
maxPCAX1P4 <- max_stable_prcomp(X1trafo, p = 4, s = s, n_initial_guesses = 15000)

In [8]:
maxPCAX1P3

$p
[1] 3

$d
[1] 5

$decoder_matrix
          [,1]      [,2]      [,3]
[1,] 0.2210190 0.2555115 1.0846493
[2,] 1.2317541 0.1919345 0.2241872
[3,] 0.2715682 1.0248831 0.2014875
[4,] 0.8459549 0.8409622 0.7489377
[5,] 0.8557159 0.8791416 0.6784372

$encoder_matrix
          [,1]      [,2]      [,3]      [,4]      [,5]
[1,] 0.1380066 0.8118467 0.1546157 0.2023784 0.2676179
[2,] 0.1258221 0.1903093 0.9757167 0.2566349 0.4108657
[3,] 0.9219583 0.2201631 0.2347719 0.1040156 0.2126178

$reconstr_matrix
          [,1]      [,2]      [,3]      [,4]      [,5]
[1,] 1.0000015 0.2387997 0.2546452 0.1128204 0.2306157
[2,] 0.2066913 0.9999955 0.1904485 0.2492804 0.3296394
[3,] 0.1857630 0.2204718 0.9999955 0.2630207 0.4210893
[4,] 0.6904894 0.6867857 0.8205408 0.2158202 0.3455225
[5,] 0.6254908 0.6947101 0.8577931 0.2256184 0.3612091

$loss_fctn_value
[1] 0.5612419

$optim_conv_status
[1] 4

$s
[1] 250

$starting_vals
$starting_vals$decoder_matrix_x0
          [,1]      [,2]      [,3]
[1,] 0.2210190 

In [9]:
estimate_dist <- function(maxpca, n, d, encoded_extr_data) {
    result <- maxpca$loss_fctn_value 
    getxstar <- function(A, b) {
        res <- rep(0, maxpca$p)
        for(j in 1:(maxpca$p)){
            res[j] <- min(b / A[,j])
        } 
        return(res)
    }
                                    
    # determine relevant indices that form basis 
    rowmax <- apply(maxpca$encoder_matrix, 1, max)
    inds <- c()
    for(i in 1:maxpca$p) {
        inds <- c(inds, which(maxpca$encoder_matrix[i,] == rowmax[i])) 
    }
                  
    datinds <- 1:d
    notactiveinds <- datinds[which(!(datinds %in% inds))]
                  
    residmat <- matrix(NA, d - maxpca$p, maxpca$p)
    
    ind <- 1
    for(i in notactiveinds) {
        tmpxstar <- getxstar(maxpca$decoder_matrix[inds, ], maxpca$decoder_matrix[i,])
        tmpapprox <- maxmatmul(maxpca$decoder_matrix[inds,], tmpxstar)
        residmat[ind,] <- maxpca$decoder_matrix[i,] - tmpapprox
        ind <- ind + 1
    }
    if(maxpca$p == 2) return(result)
    
    end <- apply(encoded_extr_data, 1, function(z) z / sum(z))
    return(result + maxpca$s / n * sum(maxmatmul(residmat, end)))
}

In [10]:
estimate_dist(maxPCAX1P3, 10000, 5, t(maxmatmul(maxPCAX1P3$encoder_matrix, t(X1trafo)))[which(rowSums(X1trafo) > s), ])

[1] 0.5612419

In [11]:
round(maxPCAX1P3$decoder_matrix, 2)

0.22,0.26,1.08
1.23,0.19,0.22
0.27,1.02,0.20
0.85,0.84,0.75
0.86,0.88,0.68


In [12]:
round(maxPCAX1P3$encoder_matrix, 2)

0.14,0.81,0.15,0.20,0.27
0.13,0.19,0.98,0.26,0.41
0.92,0.22,0.23,0.10,0.21


In [13]:
round(maxPCAX1P3$reconst_matrix, 2)

ERROR: Error in round(maxPCAX1P3$reconst_matrix, 2): non-numeric argument to mathematical function


In [ ]:
maxPCAX1P1$loss_fctn_value

### elbow plot and pairplot to visually inspect fit

In [ ]:
plotlossdatX1 <- data.frame(
    p = 1:4, 
    loss = c(
        maxPCAX1P1$loss_fctn_value, 
        maxPCAX1P2$loss_fctn_value, 
        maxPCAX1P3$loss_fctn_value, 
        maxPCAX1P4$loss_fctn_value

    )
)

pe1 <- ggplot(aes(x = p, y = loss), data = plotlossdatX1) + geom_line() + geom_point() + ylim(0,4) + theme_minimal()
pe1

In [ ]:
ggsave("logistic_dephigh_elbow.pdf", pe1)

In [ ]:
rec1 <- t(maxmatmul(maxPCAX1P3$reconstr_matrix, t(exp(X1))))

In [ ]:
datX1 <- data.frame(exp(X1))
datX1$is_rec = "original"
datrec1 <- data.frame(rec1)
datrec1$is_rec = "reconstruction"
names(datrec1) <- names(datX1)
plot_datX1 <- full_join(datX1, datrec1)

In [ ]:
p1 <- ggpairs(
    plot_datX1, 
    aes(color = is_rec), 
    columns = 1:5,
    upper = list(continuous = "points", combo = "dot_no_facet"),
    diag = list(continuous = "blankDiag", discrete = "barDiag", na = "naDiag")
) + 
theme_light() +
theme(axis.text.x = element_text(angle = 45, vjust = 1, hjust=1))

p1

In [ ]:
ggsave("logistic_dephigh.png", p1)

# Max-stable PCA for X2

In [ ]:
length(which(rowSums(X2trafo) > s))

In [ ]:
maxPCAX2P1 <- max_stable_prcomp(X2trafo, p = 1, s = s, n_initial_guesses = 15000)
maxPCAX2P2 <- max_stable_prcomp(X2trafo, p = 2, s = s, n_initial_guesses = 15000)
maxPCAX2P3 <- max_stable_prcomp(X2trafo, p = 3, s = s, n_initial_guesses = 15000)
maxPCAX2P4 <- max_stable_prcomp(X2trafo, p = 4, s = s, n_initial_guesses = 15000)

In [ ]:
maxPCAX2P3

In [ ]:
estimate_dist(maxPCAX2P3, 10000, 5, t(maxmatmul(maxPCAX2P3$encoder_matrix, t(X2trafo)))[which(rowSums(X2trafo) > s), ])

In [ ]:
round(maxPCAX2P3$decoder_matrix, 2)

In [ ]:
round(maxPCAX2P3$encoder_matrix, 2)

In [ ]:
round(maxPCAX2P3$reconstr_matrix, 2)

### elbow plot and pairplot to visually inspect fit

In [ ]:
plotlossdatX2 <- data.frame(
    p = 1:4, 
    loss = c(
        maxPCAX2P1$loss_fctn_value,
        maxPCAX2P2$loss_fctn_value,
        maxPCAX2P3$loss_fctn_value,
        maxPCAX2P4$loss_fctn_value

    )
)

pe2 <- ggplot(aes(x = p, y = loss),  data = plotlossdatX2) + geom_line() + geom_point() + ylim(0,4) + theme_minimal()
pe2

In [ ]:
ggsave("logistic_depmed_elbow.pdf", pe2)

In [ ]:
rec2 <- t(maxmatmul(maxPCAX2P3$reconstr_matrix, t(exp(X2))))

In [ ]:
datX2 <- data.frame(exp(X2))
datX2$is_rec = "original"
datrec2 <- data.frame(rec2)
datrec2$is_rec = "reconstruction"
names(datrec2) <- names(datX2)
plot_datX2 <- full_join(datX2, datrec2)

In [ ]:
p2 <- ggpairs(
    plot_datX2, 
    aes(color = is_rec), 
    columns = c(1,3,4,2,5),
    upper = list(continuous = "points", combo = "dot_no_facet"),
    diag = list(continuous = "blankDiag", discrete = "barDiag", na = "naDiag")
) + 
theme_light() +
theme(axis.text.x = element_text(angle = 45, vjust = 1, hjust=1))

p2

In [ ]:
ggsave("logistic_depmed.png", p2)

# Same for X3

In [ ]:
length(which(rowSums(X3trafo) > s))

In [ ]:
maxPCAX3P1 <- max_stable_prcomp(X3trafo, p = 1, s = s, n_initial_guesses = 15000)
maxPCAX3P2 <- max_stable_prcomp(X3trafo, p = 2, s = s, n_initial_guesses = 15000)
maxPCAX3P3 <- max_stable_prcomp(X3trafo, p = 3, s = s, n_initial_guesses = 15000)
maxPCAX3P4 <- max_stable_prcomp(X3trafo, p = 4, s = s, n_initial_guesses = 15000)

In [ ]:
maxPCAX3P3

In [ ]:
estimate_dist(maxPCAX3P3, 10000, 5, t(maxmatmul(maxPCAX3P3$encoder_matrix, t(X3trafo)))[which(rowSums(X3trafo) > s), ])

In [ ]:
round(maxPCAX3P3$decoder_matrix, 2)

In [ ]:
round(maxPCAX3P3$encoder_matrix, 2)

### elbow plot and pairplot to visually inspect fit

In [ ]:
plotlossdatX3 <- data.frame(
    p = 1:4, 
    loss = c(
        maxPCAX3P1$loss_fctn_value,
        maxPCAX3P2$loss_fctn_value,
        maxPCAX3P3$loss_fctn_value,
        maxPCAX3P4$loss_fctn_value

    )
)

pe3 <- ggplot(aes(x = p, y = loss),  data = plotlossdatX3) + geom_line() + geom_point() + ylim(0,4) + theme_minimal()
pe3

In [ ]:
ggsave("logistic_deplow_elbow.pdf", pe3)

In [ ]:
rec3 <- t(maxmatmul(maxPCAX3P3$reconstr_matrix, t(exp(X3))))

In [ ]:
datX3 <- data.frame(exp(X3))
datX3$is_rec = "original"
datrec3 <- data.frame(rec3)
datrec3$is_rec = "reconstruction"
names(datrec3) <- names(datX3)
plot_datX3 <- full_join(datX3, datrec3)

In [ ]:
p3 <- ggpairs(
    plot_datX3, 
    aes(color = is_rec), 
    columns = c(2,3,5,1,4),
    upper = list(continuous = "points", combo = "dot_no_facet"),
    diag = list(continuous = "blankDiag", discrete = "barDiag", na = "naDiag")
) + 
theme_light() +
theme(axis.text.x = element_text(angle = 45, vjust = 1, hjust=1))

p3

In [ ]:
ggsave("logistic_deplow.png", p3)